In [ ]:
!pip install texthero
# to convert MSword doc to txt for processing.
!pip install docx2txt


import pandas as pd
import numpy as np
import ast

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk import pos_tag
from nltk import word_tokenize
from nltk.stem import PorterStemmer

import re
import texthero as hero
from bs4 import BeautifulSoup
import unicodedata

import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
import docx2txt

from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from gensim.models.phrases import Phrases, Phraser
from copy import deepcopy

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer

from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances

from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams

from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords# Import the library


In [ ]:
# !pip install fsspec==0.7.4 
basefn = "C://Users//valli//Google Drive//Dell//Semester 6//Capstone//Data//"
jobstreet = pd.read_csv(basefn + "jobstreet2020-09-28.csv")

In [ ]:
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

In [ ]:
# Convert the string literal of dictionary into dictionary
def str_to_dic(x):
    try:
        new = ast.literal_eval(x)['label']
        return new
    except:
        pass

In [ ]:
jobstreet['Job Description'] = jobstreet['Job Description'].apply(lambda x: striphtml(str(x)))
jobstreet = jobstreet.dropna(subset=['Job Description'])

jobstreet['Industry'] = jobstreet['Industry'].apply(lambda x: str_to_dic(x))

In [ ]:
jobstreet['Job Description'] 

# Basic NLP stuff

In [ ]:
jobs = jobstreet.iloc[:, [1,2,4]]

In [ ]:
jobs['Job Description'] = hero.remove_stopwords(jobs['Job Description'])
jobs['Job Description'] = hero.remove_urls(jobs['Job Description'])
jobs['Job Description'] = hero.tokenize(jobs['Job Description']).astype(str)
jobs['Job Description'] = hero.clean(jobs['Job Description'])
# stemmer = SnowballStemmer('english')
# jobs['stemmed jd'] = jobs['Job Description'].apply(lambda x: [stemmer.stem(y) for y in x])


In [ ]:
jobs['Job Description']

In [ ]:
top_20 = hero.visualization.top_words(jobs.loc[:, 'Job Description']).head(20)
top_20.plot.bar(rot=90, title="Top 20 words in corpus");
plt.show(block=True)
#to be added to the stopwords

In [ ]:
hero.wordcloud(jobs.loc[:, 'Job Description'], max_words=100)
#to be added to the stopwords

In [ ]:
jobs.loc[jobs['Job Description'].str.contains('money', na=False)].head(3)


In [ ]:
# searchfor = ['enron stock', 'sell stock', 'stock bonus', 'sell enron stock']

# filtered_emails  = df_corpus.loc[df_corpus['content'].str.contains('|'.join(searchfor), na=False)]
# filtered_emails.head(2)

In [ ]:
jobs 

In [ ]:
jobs = jobs.drop_duplicates('Job Description')
jobs = jobs.dropna(subset=['Industry', 'Job Title'])  # take out job title

In [ ]:
industries = jobs['Industry'].value_counts().index.tolist()
fig, ax = plt.subplots(figsize = (25, 10))
sns.countplot(x = jobs['Industry'], order = industries, ax = ax)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
industries = jobs['Industry'].value_counts().index.tolist()
fig, ax = plt.subplots(figsize = (25, 10))
sns.countplot(x = jobs['Industry'], order = industries[1:], ax = ax)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
len(jobs['Industry'].unique())

In [ ]:
jobs['Industry'].unique()

In [ ]:
jobs_revised = jobs[jobs["Industry"] != "Human Resources Management / Consulting"]

## Vectorising, clustering, heatmap

https://www.kaggle.com/kitakoj18/exploring-wine-descriptions-with-nlp-and-kmeans

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(jobs_revised["Job Description"].values)

In [ ]:
word_features = vectorizer.get_feature_names()
word_features[100:200]


In [ ]:
kmeans = KMeans(n_clusters = 10, n_init = 5, n_jobs = -1)
kmeans.fit(X)

#stem and lemmatise
# topic modelling vs clustering

In [ ]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(word_features[word] for word in centroid))


In [ ]:
jobs_revised['cluster'] = kmeans.labels_

In [ ]:
clusters = jobs_revised.groupby(['cluster', 'Industry']).size()
fig2, ax2 = plt.subplots(figsize = (30, 15))
sns.heatmap(clusters.unstack(level = 'Industry'), ax = ax2, cmap = 'Reds')

ax2.set_xlabel('Industry', fontdict = {'weight': 'bold', 'size': 24})
ax2.set_ylabel('cluster', fontdict = {'weight': 'bold', 'size': 24})
for label in ax2.get_xticklabels():
    label.set_size(16)
    label.set_weight("bold")
for label in ax2.get_yticklabels():
    label.set_size(16)
    label.set_weight("bold")

In [ ]:
jobs_selected = jobs_revised[jobs_revised["Industry"].isin(["Consulting (IT, Science, Engineering & Technical)", 'Non-Profit Organisation / Social Services / NGO', 'Healthcare / Medical',  'Computer / Information Technology (Software)', 'Banking / Financial Services', 'BioTechnology/Pharmaceutical/Clinical research', 'Computer/Information Technology (Hardware)'  ]) ]

In [ ]:
vectorizer = TfidfVectorizer()
X2 = vectorizer.fit_transform(jobs_selected["Job Description"].values)
word_features = vectorizer.get_feature_names()

kmeans = KMeans(n_clusters = 4, n_jobs = -1)
kmeans.fit(X2)

# plt.scatter(X2[:, 0], X2[:, 1], c=prediction, s=50, cmap='viridis')
# centers = k.means.cluster_centers_
# plt.scatter(centers[:, 0], centers[:, 1],c='black', s=300, alpha=0.6)

common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(word_features[word] for word in centroid))
    
jobs_selected['cluster'] = kmeans.labels_

clusters = jobs_selected.groupby(['cluster', 'Industry']).size()
fig2, ax2 = plt.subplots(figsize = (30, 15))
sns.heatmap(clusters.unstack(level = 'Industry'), ax = ax2, cmap = 'Reds')

ax2.set_xlabel('Industry', fontdict = {'weight': 'bold', 'size': 24})
ax2.set_ylabel('cluster', fontdict = {'weight': 'bold', 'size': 24})
for label in ax2.get_xticklabels():
    label.set_size(16)
    label.set_weight("bold")
for label in ax2.get_yticklabels():
    label.set_size(16)
    label.set_weight("bold")
    
# Number of rows of data

https://towardsdatascience.com/k-means-clustering-8e1e64c1561c

In [ ]:
tf_idf_norm = normalize(X)
tf_idf_array = tf_idf_norm.toarray()

In [ ]:
tf_idf_array

In [ ]:
sklearn_pca = PCA(n_components = 2)
Y_sklearn = sklearn_pca.fit_transform(tf_idf_array)
kmeans = KMeans(n_clusters=4, max_iter=600, algorithm = 'auto')
fitted = kmeans.fit(Y_sklearn)
prediction = kmeans.predict(Y_sklearn)
plt.scatter(Y_sklearn[:, 0], Y_sklearn[:, 1], c=prediction, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1],c='black', s=300, alpha=0.6)

In [ ]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(word_features[word] for word in centroid))

Need to work on extracting the words

In [ ]:
number_clusters = range(1, 10)

kmeans = [KMeans(n_clusters=i, max_iter = 600) for i in number_clusters]
kmeans

score = [kmeans[i].fit(Y_sklearn).score(Y_sklearn) for i in range(len(kmeans))]
score

plt.plot(number_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Method')
plt.show()

http://localhost:8888/notebooks/Downloads/NLP/11_Word2Vec_via%20Gensim.ipynb

## Word2Vec

In [ ]:
jobD = [row.split() for row in jobs['Job Description']]
phrases = Phrases(jobD, min_count=10, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[jobD]


In [ ]:
cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(min_count=10,
                     window=3,
                     size=55,
                     sample=6e-5, 
                     alpha=0.05, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
w2v_model.build_vocab(sentences, progress_per=10000)

In [ ]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=40, report_delay=1)

In [ ]:
w2v_model.wv.most_similar(positive=["health"])

In [ ]:
w2v_model.wv.most_similar(positive=["finance"])

In [ ]:
w2v_model.save('skills_model')

In [ ]:
w2v_model.wv.doesnt_match(['bio', 'medicine', 'computer'])

In [ ]:
w2v_model.wv.most_similar(positive=["software", "hardware"], negative=["number"], topn=3)

In [ ]:
vocabs = []
vecs = []
for word in w2v_model.wv.vocab:
    vocabs.append(word)
    vecs.append(w2v_model[word])

In [ ]:
sub_vecs = vecs[:100]
sub_vocab = vocabs[:100]
tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2000, random_state=23)
new_values = tsne_model.fit_transform(sub_vecs)

In [ ]:
%matplotlib inline

x = []
y = []
for value in new_values:
    x.append(value[0])
    y.append(value[1])
        
plt.figure(figsize=(16, 16)) 
for i in range(len(x)):
    plt.scatter(x[i],y[i])
    plt.annotate(sub_vocab[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
plt.show()

### Resume Matching - Medium

In [ ]:
gojek = "Perform analysis, dive deep into data sets, and build reports/dashboards as needed in order to monitor business health and solve business problems. \
Be a driver of action to identify valuable business insights to further scale our business in Indonesia and operations in other countries. \
Evaluate the many iterative experiments we run regularly and their impact to our North Star Metric. \
Be a trusted business advisor, offering guidance in how to most effectively utilize data to solve issues (not just simply responding to requests). \
Engage in hypothesis-driven problem solving, and build processes tools that'll enable your teammates to easily do the same. \
Suggest A/B tests and experimentation for product features; analyze how the key metrics respond and evaluate; guide decisions on whether to increase exposure or stop the experiment. \
Engage and partner with various Product & Engineering teams to conduct research in efforts to improve our business. \
Currently in your third or final year of studies, majoring in either computer science, software engineering, and other technical fields. \
Basic knowledge and experience working with SQL. \
Deep curiosity, able to learn independently, and work well in a team. \
Strong written and verbal English communication abilities. \
Prior experience working on analytical projects. \
Able to write in Python/R and visualize analysis using Tableau/Python. \
Basic knowledge in statistics. \
A keen eye for patterns in data."

#Summarize the text with ratio 0.1 (10% of the total words.)
# text_resume = summarize(text_resume, ratio=0.1)


In [ ]:
shopee = "This opportunity is only for students who are available for a minimum of 12 weeks in the period of May - August 2021. \
You can now apply for our 2021 Data Analytics & Data Science Summer Internships , running from May to August. \
Our internships are designed to provide real-world experience that enables you to put everything you've learned into action and give you an opportunity to join us in driving South East Asia forward, together. \
We strive to make a difference and disrupt the world of transport and financial services.\
Full-time student majoring in Computer Sciences, Engineering, Statistics, Data Science or related fields seeking a matriculated internship\
You are enrolled in a Singapore university, or Singaporean/PR studying abroad\
Previous internship experience in Data Analytics, Computer Science or other related field is an advantage\
Be able to commit full-time from May toAugust 2021, for a minimum of 12 weeks \
Agile and able to work in a fast-paced environment\
Excellent communication, presentation and project management skills \
We require our interns in Singapore to be covered under the Employment Act (EA) and Work Injury Compensation (WICA) act. As such, our internship opportunities are limited to students of any nationality studying in Singapore, and Singaporean/PR studying in any location. \
For more information, view the Ministry of Manpower (MOM) website. If you do not qualify for our internship opportunities, we encourage you to review our Traineeship openings instead!\
Ready to apply? Please submit a resume and cover letter by 31st January 2021 indicating your area of interest from the above mentioned opportunities. We look forward to speaking to you soon!\
Get to know Grab: \
Grab is more than just the leading ride-hailing and mobile payments platform in Southeast Asia. We use data and technology to improve everything from transportation to payments and financial services across a region of more than 620 million people. We work with governments, drivers, passengers, merchants, and the community, to solve critical problems in Southeast Asia. \
Grab began as a taxi-hailing app in 2012, but we have since extended our product platform to include GrabCar, GrabShare, GrabBike, GrabHitch, GrabExpress, GrabFood, GrabCoach, GrabShuttle, GrabCycle. We recently launched our fintech platform – GrabFinancial, which consists of payments, lending and insurance. Our latest addition is GrabVentures, an in-house incubation platform. We are focused on pioneering new commuting and payment alternatives for drivers and passengers with an emphasis on convenience, safety, and reliability. Currently, we offer services in 8 countries. Our R&D offices are in Singapore, Seattle, Beijing, Bangalore, Jakarta and Vietnam. We aspire to unlock the true potential of Southeast Asia and look for like-minded individuals to join us on this ride. \
If you share our vision of driving South East Asia forward, apply to join our team today."
# text = summarize(text, ratio=0.1)

In [ ]:
text_list = [gojek, shopee]
cv = CountVectorizer()
count_matrix = cv.fit_transform(text_list)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# get the match percentage
matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
matchPercentage = round(matchPercentage, 2) # round to two decimal
print("Your resume matches about " + str(matchPercentage)+ "% of the job description.")


## Needa try out

In [ ]:
bigram=list(ngrams(Sentences.lower().split(),2))
trigram=list(ngrams(Sentences.lower().split(),3))
fourgram=list(ngrams(Sentences.lower().split(),4))

# Unneccessary stuff


In [ ]:
exit= False
for row in range(len(df_corpus)):
    if ):
        continue
    else:
        for i in range(10, 47):
            if df_corpus.iloc[row, i][0] == "{":
                df_corpus.iloc[row, 2] = df_corpus.iloc[row, i][0]
                #df_corpus.iloc[row, 1] = [' '.join(row.astype(str)) for row in df_corpus[df_corpus.columns[1:i-1]].values]
                break
            break
    

        
    

In [ ]:
class Kmeans:
    """ K Means Clustering
    
    Parameters
    -----------
        k: int , number of clusters
        
        seed: int, will be randomly set if None
        
        max_iter: int, number of iterations to run algorithm, default: 200
        
    Attributes
    -----------
       centroids: array, k, number_features
       
       cluster_labels: label for each data point
       
    """
    
    def __init__(self, k, seed = None, max_iter = 200):
        self.k = k
        self.seed = seed
        if self.seed is not None:
            np.random.seed(self.seed)
        self.max_iter = max_iter



    def initialise_centroids(self, data):
        """Randomly Initialise Centroids

        Parameters
        ----------
        data: array or matrix, number_rows, number_features

        Returns
        --------
        centroids: array of k centroids chosen as random data points 
        """

        initial_centroids = np.random.permutation(data.shape[0])[:self.k]
        self.centroids = data[initial_centroids]

        return self.centroids

    def predict(self, data):
        """Predict which cluster data point belongs to

        Parameters
        ----------
        data: array or matrix, number_rows, number_features

        Returns
        --------
        cluster_labels: index which minmises the distance of data to each
        cluster
        """
    
        return self.assign_clusters(data)

    def assign_clusters(self, data):
        """Compute distance of data from clusters and assign data point
           to closest cluster.

        Parameters
        ----------
        data: array or matrix, number_rows, number_features

        Returns
        --------
        cluster_labels: index which minmises the distance of data to each
        cluster

        """
        

        if data.ndim == 1:
            data = data.reshape(-1, 1)

        dist_to_centroid =  pairwise_distances(data, self.centroids, metric = 'euclidean')
        self.cluster_labels = np.argmin(dist_to_centroid, axis = 1)

        return  self.cluster_labels


    def update_centroids(self, data):
        """Computes average of all data points in cluster and
           assigns new centroids as average of data points

        Parameters
        -----------
        data: array or matrix, number_rows, number_features

        Returns
        -----------
        centroids: array, k, number_features
        """

        self.centroids = np.array([data[self.cluster_labels == i].mean(axis = 0) for i in range(self.k)])

        return self.centroids
    

    def fit_kmeans(self, data):
        """
        This function contains the main loop to fit the algorithm
        Implements initialise centroids and update_centroids
        according to max_iter
        -----------------------

        Returns
        -------
        instance of kmeans class

        """
        
        self.centroids = self.initialise_centroids(data)

        # Main kmeans loop
        for iter in range(self.max_iter):

            self.cluster_labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)          
            if iter % 100 == 0:
                print("Running Model Iteration %d " %iter)
        print("Model finished running")
        return self    